## Qwen-Image & Qwen-Image Edit

In [ ]:
import diffusers

# if not hasattr(diffusers.utils.constants, "MIN_PEFT_VERSION"):
# diffusers.utils.constants.MIN_PEFT_VERSION = "0.7.0"  # PEFT最低版本要求
from diffusers import DiffusionPipeline
import torch

model_name = "Qwen/Qwen-Image"
model_name = "your_model_path"  
# Load the pipeline
if torch.cuda.is_available():
    torch_dtype = torch.bfloat16
    device = "cuda"
else:
    torch_dtype = torch.float32
    device = "cpu"

pipe = DiffusionPipeline.from_pretrained(model_name, torch_dtype=torch_dtype)
pipe = pipe.to(device)


In [ ]:
model = pipe.transformer
total = 0
for name, p in model.named_parameters():
    total += p.numel() 

total / 1e9

In [ ]:
import torch
import torch.nn as nn

sparse_ratio = 0.3

def count_parameters(model, only_trainable=False, exclude_label=None):
    """
    统计模型参数量
    Args:
        model: nn.Module, 模型
        only_trainable: bool, 是否只统计可训练参数
        exclude_label: list[str], 要排除的参数名关键词
    Returns:
        参数总数 (int)
    """
    if exclude_label is None:
        exclude_label = []

    total = 0
    for name, p in model.named_parameters():
        if any(label in name for label in exclude_label):
            continue
        if only_trainable and not p.requires_grad:
            continue
        print(name)
        total += p.numel() * sparse_ratio if "mlp" in name else p.numel()
    return total


exclude_label = ["moe_gen", "vit_model", "connector", "vae2llm", "llm2vae", "time_embedder", "pos_embed"]
exclude_label = ["visual"]

model = pipe.text_encoder
print("Total Parameters:", count_parameters(model, exclude_label=exclude_label) /  1e9)

In [ ]:
from diffusers.pipelines.qwenimage.modeling_qwen2_5_vl import Qwen2_5_VLForConditionalGeneration

model_path = f"{model_name}/text_encoder"
pipe.text_encoder = Qwen2_5_VLForConditionalGeneration.from_pretrained(
    model_path, torch_dtype="auto", device_map="auto", trust_remote_code=True, 
).to(torch_dtype)

In [ ]:

positive_magic = {
    "en": ", Ultra HD, 4K, cinematic composition.", # for english prompt
    "zh": ", 超清，4K，电影级构图." # for chinese prompt
}

# Generate image
prompt = '''A coffee shop entrance features a chalkboard sign reading "Qwen Coffee 😊 $2 per cup," with a neon light beside it displaying "通义千问". Next to it hangs a poster showing a beautiful Chinese woman, and beneath the poster is written "π≈3.1415926-53589793-23846264-33832795-02384197". Ultra HD, 4K, cinematic composition'''
negative_prompt = " " # using an empty string if you do not have specific concept to remove

# Generate with different aspect ratios
aspect_ratios = {
    "1:1": (1328, 1328),
    "16:9": (1664, 928),
    "9:16": (928, 1664),
    "4:3": (1472, 1140),
    "3:4": (1140, 1472),
    "3:2": (1584, 1056),
    "2:3": (1056, 1584),
}

width, height = aspect_ratios["16:9"]
num_inference_steps = 2

outputs = pipe(
    prompt=prompt + positive_magic["en"],
    negative_prompt=negative_prompt,
    width=width,
    height=height,
    num_inference_steps=num_inference_steps,
    true_cfg_scale=4.0,
    generator=torch.Generator(device="cuda").manual_seed(42)
)

image = outputs.images[0]
display(image)